Load the libraries

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata.csv')

In [ ]:
dataset.sample(3)

,Age,Salary,Purchased
25,43,52000,1
14,32,45000,0
22,40,60000,1


In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Convert training and test data to tensor format of data type float

In [ ]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()


In [ ]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [ ]:
Xtrain_.shape, ytrain_.shape

(torch.Size([32, 2]), torch.Size([32]))

In [ ]:
Xtest_.shape, ytest_.shape

(torch.Size([8, 2]), torch.Size([8]))

In [ ]:
input_size=2
output_size=2
hidden_size=10

Classifier model with 2 hidden layers. It contains linear layers and sigmoid activation function

log softmax for the classifier


In [ ]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.sigmoid((self.fc1(X)))
       X = torch.sigmoid((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [ ]:
model = Net()



learning rate = 0.01 and NLLLoss function

In [ ]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [ ]:
epochs = 100


In [ ]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.6984546780586243
Epoch 1 loss 0.6883957982063293
Epoch 2 loss 0.6822064518928528
Epoch 3 loss 0.6791566610336304
Epoch 4 loss 0.6778274178504944
Epoch 5 loss 0.67659991979599
Epoch 6 loss 0.6745062470436096
Epoch 7 loss 0.6713821887969971
Epoch 8 loss 0.6674991846084595
Epoch 9 loss 0.6632304191589355
Epoch 10 loss 0.6588670611381531
Epoch 11 loss 0.6545367240905762
Epoch 12 loss 0.6501933932304382
Epoch 13 loss 0.6456645727157593
Epoch 14 loss 0.6407316327095032
Epoch 15 loss 0.6352043151855469
Epoch 16 loss 0.6289618015289307
Epoch 17 loss 0.621955156326294
Epoch 18 loss 0.6141886711120605
Epoch 19 loss 0.6056920886039734
Epoch 20 loss 0.5965004563331604
Epoch 21 loss 0.5866397619247437
Epoch 22 loss 0.5761235356330872
Epoch 23 loss 0.5649536848068237
Epoch 24 loss 0.5531277060508728
Epoch 25 loss 0.5406484603881836
Epoch 26 loss 0.5275341272354126
Epoch 27 loss 0.5138262510299683
Epoch 28 loss 0.49959197640419006
Epoch 29 loss 0.4849202036857605
Epoch 30 loss 0.469916

model paramters can be saved

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1528,  1.1299],
         [-1.3217, -1.2464],
         [ 0.9343,  1.5205],
         [ 0.8340,  1.5981],
         [ 1.1413,  0.3927],
         [-0.9755, -0.6465],
         [-1.0519, -1.1970],
         [-1.3397, -0.6035],
         [-0.8232, -1.4804],
         [-0.1030, -1.4247]], requires_grad=True), Parameter containing:
 tensor([-0.0473,  0.1453, -0.2659, -0.2591, -0.2823,  0.2346,  0.1407,  0.2148,
          0.2586,  0.0946], requires_grad=True), Parameter containing:
 tensor([[ 1.0649, -0.9615,  0.6178,  0.8507,  0.4290, -0.7251, -0.6516, -0.6319,
          -0.7880, -0.8869],
         [ 0.4954, -0.6788,  0.5989,  0.7300,  0.2990, -0.5910, -0.6517, -0.8506,
          -0.6689, -0.7390],
         [ 0.5259, -0.6031,  0.5900,  0.4257,  0.3482, -0.3278, -0.8952, -0.7045,
          -0.8022, -0.4769],
         [ 0.6720, -0.5637,  0.6571,  0.9568,  0.5646, -0.5459, -0.8427, -0.8755,
          -0.5685, -0.8071],
         [ 0.4090, -0.8782,  0.7230,  0.8878,  

In [ ]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[ 0.0829, -1.4975]])

In [ ]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0480, -3.0606]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [ ]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.5621, -0.2353]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

doesn't save the model class

In [ ]:
torch.save(model,'customer_buy.pt')

In [ ]:
!ls

customer_buy.pt  sample_data


In [ ]:
restored_model = torch.load('customer_buy.pt')

In [ ]:
y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0480, -3.0606]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [ ]:
y_cust_42_50000 = restored_model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.5621, -0.2353]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [ ]:
model.state_dict()

OrderedDict([('fc1.weight', tensor([[ 0.1528,  1.1299],
                      [-1.3217, -1.2464],
                      [ 0.9343,  1.5205],
                      [ 0.8340,  1.5981],
                      [ 1.1413,  0.3927],
                      [-0.9755, -0.6465],
                      [-1.0519, -1.1970],
                      [-1.3397, -0.6035],
                      [-0.8232, -1.4804],
                      [-0.1030, -1.4247]])),
             ('fc1.bias',
              tensor([-0.0473,  0.1453, -0.2659, -0.2591, -0.2823,  0.2346,  0.1407,  0.2148,
                       0.2586,  0.0946])),
             ('fc2.weight',
              tensor([[ 1.0649, -0.9615,  0.6178,  0.8507,  0.4290, -0.7251, -0.6516, -0.6319,
                       -0.7880, -0.8869],
                      [ 0.4954, -0.6788,  0.5989,  0.7300,  0.2990, -0.5910, -0.6517, -0.8506,
                       -0.6689, -0.7390],
                      [ 0.5259, -0.6031,  0.5900,  0.4257,  0.3482, -0.3278, -0.8952, -0.7045,
   

In [ ]:
optimizer.state_dict()

{'param_groups': [{'amsgrad': False,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'lr': 0.01,
   'params': [0, 1, 2, 3, 4, 5],
   'weight_decay': 0}],
 'state': {0: {'exp_avg': tensor([[ 2.1298e-04, -3.2874e-03],
           [-1.7683e-04,  5.0524e-03],
           [-1.6823e-04, -4.5844e-03],
           [-2.5593e-04, -4.9523e-03],
           [ 3.1286e-04, -3.3148e-03],
           [-2.5009e-04,  3.9614e-03],
           [-6.3593e-05,  4.5839e-03],
           [-3.8099e-04,  4.5943e-03],
           [ 1.8829e-04,  4.5881e-03],
           [-7.3209e-05,  3.8789e-03]]),
   'exp_avg_sq': tensor([[9.2455e-07, 1.3341e-06],
           [4.1795e-06, 6.0253e-06],
           [3.6355e-06, 4.2908e-06],
           [5.0300e-06, 5.3648e-06],
           [1.1294e-06, 1.9572e-06],
           [3.4590e-06, 4.5680e-06],
           [4.6408e-06, 5.6592e-06],
           [5.3616e-06, 7.3035e-06],
           [5.3601e-06, 5.5138e-06],
           [1.3435e-06, 1.8354e-06]]),
   'step': 100},
  1: {'exp_avg': tensor([ 0.00

In [ ]:
torch.save(model.state_dict(),'customer_buy_state_dict')

In [ ]:
!ls

customer_buy.pt  customer_buy_state_dict  sample_data


In [ ]:
new_predictor = Net()

<All keys matched successfully>

In [ ]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0480, -3.0606]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
y_cust_42_50000 = new_predictor(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.5621, -0.2353]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
!ls

customer_buy.pt  customer_buy_state_dict  sample_data


In [ ]:
!zip -r customer_buy_state_dict.zip customer_buy_state_dict

  adding: customer_buy_state_dict (deflated 48%)


In [ ]:
!ls


customer_buy.pt		 customer_buy_state_dict.zip
customer_buy_state_dict  sample_data


In [ ]:
from google.colab import files


In [ ]:
files.download('customer_buy_state_dict.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>